In [1]:
import pandas as pd
import numpy as np
from itertools import product
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from joblib import Parallel, delayed
import warnings, time, os

warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive

drive.mount('/gdrive')
path = '/gdrive/MyDrive/Colab Notebooks/LSTM_MFT/'

In [2]:
# --- Load and prepare your data ---
# df is your multivariate DataFrame
target_col = 'num_files_total'  # change this to your chosen column

train = pd.read_csv(path + 'training_scaled.csv', sep = "|", index_col=0, parse_dates=True)
test = pd.read_csv(path + 'testing_scaled.csv', sep = "|", index_col=0, parse_dates=True)
validate = pd.read_csv(path + 'validation_scaled.csv', sep='|', index_col=0, parse_dates=True)

train.index.freq = '5min'
test.index.freq = '5min'
validate.index.freq = '5min'

X_train = train.drop(columns=[target_col])
y_train = train[target_col]

X_test = test.drop(columns=[target_col])
y_test = test[target_col]

X_validate = validate.drop(columns=[target_col])
y_validate = validate[target_col]

In [3]:
train.head()

,num_files_total,count_moving_mean,count_moving_std,files_size_MB_total,size_moving_mean,size_moving_std
2025-06-06 00:55:00,0.000403,0.010129,0.003515,0.000406,0.049778,0.008935
2025-06-06 01:00:00,0.001103,0.009202,0.003372,0.001361,0.050744,0.008518
2025-06-06 01:05:00,0.001983,0.010351,0.003464,0.004691,0.048113,0.008032
2025-06-06 01:10:00,0.007171,0.016509,0.006634,0.005528,0.051724,0.007607
2025-06-06 01:15:00,0.006739,0.022072,0.008125,0.006166,0.050767,0.007358


In [4]:
param_grid = {
    'p': list(range(12, 73, 12)),
    'd': [0, 1], ## TODO change this back
    'q': list(range(12, 73, 12)),
    'P': [0, 1],
    'D': [0, 1],
    'Q': [0, 1],
    's': [288],
}

# Generate combinations
grid = list(ParameterGrid(param_grid))

In [5]:
# --- RESUME SUPPORT ---

csv_path = path + 'SARIMA/sarimax_gridsearch_results.csv'
result_columns = ['p', 'd', 'q', 'P', 'D', 'Q', 's',
                  'MSE', 'MAE', 'R2', 'fit_time', 'status']

if not os.path.exists(csv_path) or pd.read_csv(csv_path).empty:
    pd.DataFrame(columns=result_columns).to_csv(csv_path, index=False, mode='w')
    print(f"📄 Initializing new/empty results file with header at: {csv_path}")

if os.path.exists(csv_path):
    existing = pd.read_csv(csv_path)
    print(f"📄 Found existing results file with {len(existing)} entries. Resuming...")

    completed = set(
        tuple(row[param] for param in ['p', 'd', 'q', 'P', 'D', 'Q', 's'])
        for _, row in existing.iterrows()
    )
    grid = [
        params for params in grid
        if tuple(params[k] for k in ['p', 'd', 'q', 'P', 'D', 'Q', 's']) not in completed
    ]
    print(f"➡️  {len(grid)} parameter combinations remaining.")

📄 Found existing results file with 32 entries. Resuming...
➡️  544 parameter combinations remaining.


In [6]:
def fit_sarimax(params, y_train, y_test, X_train, X_test):
    order = (params['p'], params['d'], params['q'])
    seasonal_order = (params['P'], params['D'], params['Q'], params['s'])
    try:
        start_time = time.time()
        model = SARIMAX(
            endog=y_train,
            exog=X_train,
            order=order,
            seasonal_order=seasonal_order,
            enforce_stationarity=False,
            enforce_invertibility=False
        )
        fit = model.fit(disp=False)
        fit_time = time.time() - start_time
        
        forecast = fit.forecast(steps=len(y_test), exog=X_test)

        mse = mean_squared_error(y_test, forecast)
        mae = mean_absolute_error(y_test, forecast)
        r2 = r2_score(y_test, forecast)

        result = {
            'p': params['p'], 'd': params['d'], 'q': params['q'],
            'P': params['P'], 'D': params['D'], 'Q': params['Q'], 's': params['s'],
            'MSE': mse, 'MAE': mae, 'R2': r2, 'fit_time': fit_time, 'status': 'ok'
        }
        
        # Incremental save after each fit
        df_partial = pd.DataFrame([result])
        df_partial.to_csv(csv_path, mode='a', header=False, index=False)

        return result

    except Exception as e:
        return {
            'p': params['p'], 'd': params['d'], 'q': params['q'],
            'P': params['P'], 'D': params['D'], 'Q': params['Q'], 's': params['s'],
            'MSE': None, 'MAE': None, 'R2': None, 'fit_time': None,'status': str(e)}

In [7]:
n_jobs = 2  # use all available CPU cores (safe for Colab)
print(f"⚙️ Running grid search in parallel using {n_jobs} cores...")

# --- Parallel Execution with Incremental Saving ---
results = []
try:
    results = Parallel(n_jobs=n_jobs, verbose=10, prefer="processes")(
        delayed(fit_sarimax)(params, y_train, y_test, X_train, X_test)
        for params in grid
    )
except KeyboardInterrupt:
    print("Interrupted by user — partial results saved.")

# --- Final Save ---
if os.path.exists(csv_path):
    results_df = pd.read_csv(csv_path)
else:
    results_df = pd.DataFrame(results)
    results_df.to_csv(csv_path, index=False)

# Merge with any previous results and remove duplicates
if not existing.empty:
    results_df = pd.concat([existing, results_df]).drop_duplicates(
        subset=['p', 'd', 'q', 'P', 'D', 'Q', 's'], keep='last'
    )

results_df = results_df.sort_values('MSE', ascending=True, na_position='last')
results_df.to_csv(csv_path, index=False)

print("Búsqueda completada.")

⚙️ Running grid search in parallel using 2 cores...


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   1 tasks      | elapsed:  1.8min
/home/desktop/VENV/especializacion/.venv/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/home/desktop/VENV/especializacion/.venv/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
[Parallel(n_jobs=2)]: Done   4 tasks      | elapsed: 23.9min
/home/desktop/VENV/especializacion/.venv/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/home/desktop/VENV/especializacion/.venv/lib/py

Interrupted by user — partial results saved.
Búsqueda completada.


In [10]:
combined = pd.read_csv(csv_path)
combined = combined.sort_values('MSE')

print("Mejores resultados.")
print(results_df.head())

Mejores resultados.
    p  d   q  P  D  Q    s       MSE       MAE        R2     fit_time status
0  48  0  48  0  0  0  288  0.001851  0.012573  0.113624  2485.277780     ok
1  24  0  72  0  0  0  288  0.001851  0.012559  0.113583  6444.052182     ok
2  12  0  72  0  0  0  288  0.001851  0.012549  0.113556  5110.736380     ok
3  36  0  60  0  0  0  288  0.001851  0.012562  0.113546  3895.219617     ok
4  48  0  36  0  0  0  288  0.001852  0.012563  0.113396  1931.837415     ok


In [ ]:
best_model = combined.iloc[0].to_dict()
best_order = (best_model['p'], best_model['d'], best_model['q'])
best_s_order = (best_model['P'], best_model['D'], best_model['Q'], best_model['s'])

best_sarima = SARIMAX(endog=y_train,
                          exog=X_train,
                          order=best_order,
                          seasonal_order=best_s_order,
                          enforce_stationarity=False,
                          enforce_invertibility=False)

best_fit = best_sarima.fit(disp=False)

In [ ]:
train_forecast = best_fit.forecast(steps = len(y_train), exog=X_train)
mse_train = mean_squared_error(y_train, train_forecast)
mae_train = mean_absolute_error(y_train, train_forecast)
r2_train = r2_score(y_train, train_forecast)

print("Resultados del mejor modelo en los datos de entrenamiento mismos:")
print(f"MSE = {mse_train:.6f}\tMAE = {mae_train:.6f}\tR2 = {r2_train:.6f}.")
print(f"Proporción de errores = {(combined.iloc[0]['MSE']/mse_train):.6f}")

In [ ]:
print("Resultados del resto de modelos en los datos de entrenamiento")
for i in range(1,5):
    model = combined.iloc[i].to_dict()
    order = (model['p'], model['d'], model['q'])
    s_order = (model['P'], model['D'], model['Q'], model['s'])
    sarima = SARIMAX(endog = y_train,
                    exog = X_train,
                    order = order,
                    seasonal_order = s_order,
                    enforce_stationarity=False,
                    enforce_invertibility=False)
    train_forecast = best_fit.forecast(steps = len(y_train), exog=X_train)
    mse_train = mean_squared_error(y_train, train_forecast)
    mae_train = mean_absolute_error(y_train, train_forecast)
    r2_train = r2_score(y_train, train_forecast)

    print(f"\nModelo en la posición {i+1}:")
    print(f"MSE = {mse_train:.6f}\tMAE = {mae_train:.6f}\tR2 = {r2_train:.6f}.")
    print(f"Proporción entre las métricas = {(combined.iloc[i]['MSE']/mse_train):.6f}\n")

In [ ]:
validate_forecast = best_fit.forecast(steps = len(y_validate), exog=X_validate)
mse_validate = mean_squared_error(y_validate, validate_forecast)
mae_validate = mean_absolute_error(y_validate, validate_forecast)
r2_validate = r2_score(y_validate, validate_forecast)

print("Resultados del mejor modelo durante la validacón final:")
print(f"MSE = {mse_validate:.6f}\tMAE = {mae_validate:.6f}\tR2 = {r2_validate:.6f}.")